# Transformers

In [1]:
import pandas as pd
import numpy as np
import logic.processing as lp
import seaborn as sns

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/speroulakis/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
df = lp.load_data('drugsComTrain_raw.csv')

In [3]:
def data_filter(df,uselfCount=0,min_length=30):

    df = df.dropna()
#     condition_distribution = df['condition'].value_counts()
#     best_conditions = condition_distribution.head(top_conditions)
#     top_conditions_list = best_conditions.index.tolist()
#     df = df[df['condition'].isin(top_conditions_list)]
    df = df[df['rating'].isin([1,2,3,8,9,10])]
    df['sentiment'] = 0
    df['sentiment'] = (df['rating'].isin([8,9,10])).astype(int)
    df = df[df['usefulCount']>uselfCount]
    df['review_length'] = df['review'].apply(lambda x: len(str(x).split()))
    df = df[df.review_length>=min_length]

    return df


In [4]:
df1 = data_filter(df)

In [5]:
df2 = lp.preproc(df1)

KeyboardInterrupt: 

In [ ]:
df3 = df2.drop(columns=['drugName', 'condition', 'date', 'review_length' ])

In [ ]:
data = df3.copy()

In [ ]:
X = data['clean']
y = data['sentiment']

In [ ]:
data

In [ ]:
Xb, yb = lp.balance_dataset(X,y)

In [6]:
from transformers import AutoTokenizer
from transformers import TFAutoModel

/home/speroulakis/.pyenv/versions/3.10.6/envs/Mental_Health_Risks/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny", padding_side = "right")

In [ ]:
model = TFAutoModel.from_pretrained("prajjwal1/bert-tiny", from_pt = True)

In [ ]:
data["tokenized"] = data["review"].map(lambda x: tokenizer(x)["input_ids"])

data["len_tokenized"] = data["tokenized"].map(lambda x: len(x))

In [ ]:
sns.histplot(data['len_tokenized']);

In [ ]:
liszt = data['review'].tolist()

In [ ]:
sfg

In [ ]:
tokenized = tokenizer(liszt[:100], max_length=300, padding='max_length', truncation=True, return_tensors='tf')

In [ ]:
tokenized.get('input_ids')

In [ ]:
embed = model.predict(tokenized.get('input_ids'))

In [ ]:
embed.last_hidden_state.shape

In [ ]:
test = []
for x in range(10):
  unit = tokenized.get('input_ids')
  emb = (model.predict(unit[x:x+1])).last_hidden_state[:, 0, :]
  test.append(emb)

In [ ]:
test[0].shape

In [ ]:
seb = np.concatenate(test)

In [ ]:
seb.shape

In [8]:
import joblib
import os

In [11]:
os.getcwd()

'/home/speroulakis/code/Flotchi/Mental_Health_Risks/notebooks'

In [17]:
path = os.path.join(os.path.dirname(os.getcwd()), 'raw_data', 'final2.npy')
path

'/home/speroulakis/code/Flotchi/Mental_Health_Risks/raw_data/final2.npy'

In [16]:
f1 = np.load(path)

In [19]:
f2 = np.load(path)

In [20]:
len(f1)

48659

In [21]:
len(f2)

61231

In [22]:
f1.shape

(48659, 128)

In [26]:
f1[48658]

array([-1.31323314e+00, -1.56276450e-01, -5.49429893e+00, -1.41454530e+00,
        3.47821474e-01,  2.90152073e-01, -1.01280808e+00,  9.97111976e-01,
       -3.82509023e-01,  5.60973644e-01,  1.66516066e+00,  6.11225963e-01,
       -5.99867702e-01,  6.09384835e-01,  1.56300044e+00,  4.26805794e-01,
        4.38231587e-01,  1.04514706e+00, -7.01745987e-01, -3.52468550e-01,
        1.29967701e+00,  1.59639120e-03,  1.15911412e+00, -1.42135084e-01,
        1.75610256e+00, -1.30988050e+00, -4.07941118e-02,  1.23661179e-02,
       -4.01492089e-01, -1.15452290e+00, -7.39289165e-01, -2.05143124e-01,
       -1.02219260e+00,  5.50258994e-01, -7.47083604e-01,  8.12264681e-02,
       -4.80949581e-01, -4.69285071e-01, -2.43994999e+00,  8.12179893e-02,
        1.02070260e+00,  2.53243864e-01,  2.08897781e+00, -1.02679729e+00,
       -2.04473794e-01, -6.27262533e-01, -1.81331843e-01, -1.48347855e+00,
        4.37108159e-01, -8.24031472e-01,  9.95616376e-01,  2.88108277e+00,
        7.75416315e-01,  

In [27]:
f2[0]

array([-1.8982012 , -0.02150285, -5.4516816 , -1.7456636 ,  0.1411807 ,
        0.7573541 , -0.09095369,  0.27087227, -0.33391035, -0.15553443,
        1.2944244 ,  0.828934  ,  0.10348731,  0.33818257,  0.5690639 ,
       -0.6652243 ,  0.65383774,  0.955618  , -0.7278547 , -0.24123156,
       -0.05187121, -0.24900521,  1.5895029 ,  0.7506902 ,  2.487144  ,
       -0.96608233, -0.26743144,  0.78383416,  0.17195567, -1.4315997 ,
       -1.2832544 , -0.29915208, -1.598495  ,  0.4562462 ,  1.0463309 ,
       -1.2440773 ,  0.6342277 , -0.65271413, -1.3652346 ,  0.49837655,
        1.0855684 ,  0.08695886,  1.655322  , -1.3780081 , -0.8027005 ,
       -2.059182  ,  0.6260929 , -0.84614265,  0.13312736,  0.6490854 ,
       -0.67646563,  0.80985713, -0.0330135 ,  0.20151088,  0.97935516,
       -0.60600275, -0.3889074 , -1.3103526 , -0.19551806,  2.2284958 ,
       -0.55838263, -1.1078739 , -0.21577987, -1.0664712 , -0.46111995,
        1.5780134 ,  1.0427811 ,  0.25998017, -0.95374906,  0.57

In [30]:
merged = np.vstack((f1, f2))

In [31]:
merged.shape

(109890, 128)

In [32]:
np.save('merged.npy', merged)